In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY)

In [ ]:
from langchain import __version__ as langchain_version
import openai
print(f"LangChain version: {langchain_version}")
print("OpenAI library imported successfully.")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import SequentialChain, LLMChain, ConversationChain
from langchain.prompts import ChatPromptTemplate

In [ ]:
llm_model = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0.3, model=llm_model)

In [ ]:
summary_memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=200)

In [ ]:
order_tracking_prompt = ChatPromptTemplate.from_template(
    "Check the status of order ID {order_id}. Provide expected delivery date and any delay updates."
)

product_inquiry_prompt = ChatPromptTemplate.from_template(
    "Provide details about the product {product_name}, including specifications, pricing, and stock availability."
)

refund_policy_prompt = ChatPromptTemplate.from_template(
    "A customer wants to return an item. Based on their request: {return_request}, explain the return process and refund policy."
)

In [ ]:
order_tracking_chain = LLMChain(llm=llm, prompt=order_tracking_prompt, output_key="order_status")
product_inquiry_chain = LLMChain(llm=llm, prompt=product_inquiry_prompt, output_key="product_info")
refund_policy_chain = LLMChain(llm=llm, prompt=refund_policy_prompt, output_key="refund_details")

In [ ]:
combined_workflow = SequentialChain(
    chains=[order_tracking_chain, product_inquiry_chain, refund_policy_chain],
    input_variables=["order_id", "product_name", "return_request"],
    output_variables=["order_status", "product_info", "refund_details"],
    verbose=True
)

In [ ]:
conversation_chain = ConversationChain(llm=llm, memory=summary_memory, verbose=True)

In [ ]:
print(conversation_chain.invoke(
    "Hi, I ordered a PlayStation 5 with order ID ORD67890. It was supposed to arrive yesterday but hasn’t. Can you check the status?"
))

print(conversation_chain.invoke(
    "Also, I’m interested in buying a gaming headset. What options do you have?"
))

print(conversation_chain.invoke(
    "If I change my mind about the PlayStation 5, how can I return it, and how long does the refund take?"
))

In [ ]:
print("\nConversation Summary:", summary_memory.buffer)

In [ ]:
response = combined_workflow.invoke({
    "order_id": "ORD67890",
    "product_name": "Wireless Gaming Headset",
    "return_request": "I want to return my PlayStation 5 because I changed my mind."
})

print("\n Order Tracking Response:", response["order_status"])
print("\n Product Inquiry Response:", response["product_info"])
print("\n Refund Policy Response:", response["refund_details"])